In [112]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers

In [113]:
# Read the modified Dataframe
df = pd.read_csv('modified_train.csv')

# Read the dictionary (npy) files.
w2s = np.load('word2seq.npy',allow_pickle='TRUE').item()
s2w = np.load('seq2word.npy',allow_pickle='TRUE').item()

In [114]:
df1 = df.loc[df['Converted']!='[]']
df1.index = range(len(df1))

In [169]:
tx = np.zeros((len(df1),200))

for i in range(len(df1)):
    spl = df1.Converted[i][1:-1].split(', ')
    stck = np.stack(spl)
    tx[i][:len(stck)] = stck[:200] 

ty = np.array(df1.toxic)

In [205]:
df1.head(1)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,tokenized_sents,Converted
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation\nWhy the edits made under my usern...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [267]:
ty = np.array([[df1.toxic],[df1.severe_toxic],[df1.obscene],[df1.threat],[df1.insult],[df1.identity_hate]])
ty=ty.T
ty = ty.reshape((159562,6))

In [268]:
print(' Shape of Train-X:',tx.shape,'\n','Shape of Train-Y:',ty.shape)

 Shape of Train-X: (159562, 200) 
 Shape of Train-Y: (159562, 6)


In [273]:
model = tf.keras.Sequential()

model.add(layers.Embedding(input_dim=len(s2w)+1, output_dim=64))

model.add(layers.GRU(32, return_sequences=True))

model.add(layers.SimpleRNN(16))

model.add(layers.Dense(6, activation = 'sigmoid'))

model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, None, 64)          14075328  
_________________________________________________________________
gru_31 (GRU)                 (None, None, 32)          9408      
_________________________________________________________________
simple_rnn_29 (SimpleRNN)    (None, 16)                784       
_________________________________________________________________
dense_33 (Dense)             (None, 6)                 102       
Total params: 14,085,622
Trainable params: 14,085,622
Non-trainable params: 0
_________________________________________________________________


In [274]:
model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])
model.fit(tx, ty)

4987/4987 [==============================] - 1587s 318ms/step - loss: 0.1451 - accuracy: 0.9930


In [275]:
model.save_weights('ToxicCommentsPred2.h5')